# Process raw deid data into flat files

In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np

In [2]:
mrn_auto_map = pd.read_csv('MRN_Auto_Mapping.csv')
mrn_auto_map['PAT_MRN_ID'] = mrn_auto_map['PAT_MRN_ID'].astype(int)

exclude_patients = pd.read_csv('cancer_transplants.csv')
exclude_patients['PAT_MRN_ID'] = exclude_patients['PAT_MRN_ID'].astype(int)

EXCLUDE = exclude_patients.merge(mrn_auto_map, how='left', on='PAT_MRN_ID')
# EXCLUDE.to_csv(os.path.join(cfg.OUT_DATA_PATH, 'exclude_patients.csv'))

In [8]:
identifier_cols = ['AUTO_ID', 'RESULT_YEAR', 'RESULT_MONTH']

def identifier_prefix(cols):
    identifier_cols = ['AUTO_ID', 'RESULT_YEAR', 'RESULT_MONTH']
    return identifier_cols+[x for x in cols if x not in identifier_cols]

opj = os.path.join

# IBD Medications
- Get intervals of prescription validity for each group of drugs

In [4]:
from functools import reduce


def active_intervals(o,e):    
    start_dates = [o.iloc[0]]
    end_dates=[]
    start_loc, end_loc = 0, 0
    
    for end_tm1, start_t0 in zip(e[:-1],o[1:]):
        if end_tm1<start_t0:
            end_dates.append(end_tm1)
            start_loc=end_loc+1
            start_dates.append(start_t0)
        end_loc+=1
    end_dates.append(e.iloc[-1])
    
    start_dates = list(map(pd.Timestamp, start_dates))
    end_dates = list(map(pd.Timestamp, end_dates))    
    active_during = list(zip(start_dates, end_dates))
    return active_during


def active_prescriptions(med_a):
    drugs = ['5_ASA', 'Systemic_steroids', 'Immunomodulators', 'Psych',\
       'ANTI_TNF', 'ANTI_IL12', 'ANTI_INTEGRIN', 'Vitamin_D']
    
    # Get contiguous use info
    intervals = []
    for dr in drugs:
        df = med_a[med_a.GROUP==dr]
        if df.empty:
            intervals.append(None)
        else:
            intervals.append(active_intervals(df.ORDERING_DATE, df.END_DATE))
    # Get activity one-hot
    drug_columns=[]
    for ix,dr in enumerate(drugs):
        intvl = intervals[ix]
        drug_col = []
        
        if not intvl:            
            drug_col = pd.Series(name=dr)
        else:
            for s,e in intvl:
                drug_col.append(pd.Series(index=pd.date_range(s,e, freq='M'), name=dr).fillna(1))
            drug_col = pd.concat(drug_col)
        drug_columns.append(drug_col)
        
    return pd.concat(drug_columns, axis=1)


In [9]:
psych_med_prescrip = pd.read_csv('deid_IBD_Registry_BA1951_Medications_2018-07-05-09-47-15.csv', parse_dates=['ORDERING_DATE', 'END_DATE'])

psych_med_prescrip = psych_med_prescrip[psych_med_prescrip.THERAPEUTIC_CLASS=='PSYCHOTHERAPEUTIC DRUGS']
psych_med_prescrip = psych_med_prescrip[psych_med_prescrip.END_DATE.notnull() & psych_med_prescrip.ORDERING_DATE.notnull()] # filter no end_date rows
psych_med_prescrip['GROUP'] = 'Psych'
psych_med_prescrip = psych_med_prescrip[['AUTO_ID', 'ORDERING_DATE', 'END_DATE', 'GROUP']]


med_df = pd.read_csv('filtered_meds2018.csv', parse_dates=['ORDERING_DATE', 'END_DATE'])
med_df = med_df[['AUTO_ID', 'ORDERING_DATE', 'END_DATE', 'GROUP']]
med_df = med_df[med_df.END_DATE.notnull() & med_df.ORDERING_DATE.notnull()] # filter no end_date rows
med_df = pd.concat([med_df, psych_med_prescrip])


# drop redundant rows like (Apr-Dec, Apr-Nov) and (Jan-Sept, Mar-Sept) ==> second row is redundant in both tuples 
med_df = med_df.sort_values(['AUTO_ID', 'ORDERING_DATE', 'END_DATE', 'GROUP'], ascending=[True, True, False, True]).drop_duplicates(['AUTO_ID', 'ORDERING_DATE', 'GROUP'])
med_df = med_df.sort_values(['AUTO_ID', 'ORDERING_DATE', 'END_DATE', 'GROUP'], ascending=[True, True, False, True]).drop_duplicates(['AUTO_ID', 'END_DATE', 'GROUP'])
# med_df.to_csv(os.path.join(cfg.OUT_DATA_PATH, 'all_rx.csv'))

med_df2 = med_df.groupby('AUTO_ID').apply(active_prescriptions).reset_index()
med_df2['RESULT_YEAR'] = med_df2.level_1.dt.year
med_df2['RESULT_MONTH'] = med_df2.level_1.dt.month
med_df2 = med_df2[identifier_cols+list(med_df.GROUP.unique())]
med_df2.to_csv('flatfiles/rx_long.csv', index=False)

# Labs
- No imputation
- Get range, numeric reading, and High/Normal/Low flags

In [26]:
labs = pd.read_csv('filtered_labs.csv', parse_dates=['ORDER_DATE','RESULT_DATE'])
labs = labs[['AUTO_ID', 'ORDER_DATE', 'RESULT_DATE', 'ORD_NUM_VALUE', 'REF_LOW', 'REF_HIGH', 'REF_NORMAL_VALS', 'REF_UNIT',
       'RESULT_FLAG', 'GROUP']]

# drop invalid rows
labs = labs.drop(labs.index[np.logical_or(labs['ORD_NUM_VALUE'].isnull(), labs['ORD_NUM_VALUE']>=9999999)]) # Drop rows with invalid readings

# === clean messy readings data ====
# copy REF_LOW values with <,- to REF_NORMAL_VALS. WILL PROCESS THEM 
labs['REF_NORMAL_VALS'] = np.where(labs['REF_LOW'].str.contains(r'[<-]')==True, labs['REF_LOW'], labs['REF_NORMAL_VALS'])
# replace invalid values with nan
labs['REF_HIGH'] = np.where(labs['REF_HIGH'].str.contains(r'[a-zA-Z<]')==True, np.nan, labs['REF_HIGH'])
labs['REF_LOW'] = np.where(labs['REF_LOW'].str.contains(r'[a-zA-Z<\-]')==True, np.nan, labs['REF_LOW'])
# drop garbage rows
labs.drop(labs[labs['REF_NORMAL_VALS'].str.contains(r'[a-zA-Z]')==True].index, inplace=True)
# extract LOW, HIGH by splitting REF_NORMAL_VALS
labs[['NEW_LOW','NEW_HIGH']] = labs[labs['REF_NORMAL_VALS'].str.contains('-')==True]['REF_NORMAL_VALS'].str.split('-').apply(pd.Series, 1) # solve cases where REF_LOW is like 0-10
labs['NEW_LOW'] = np.where(np.logical_and(labs['REF_LOW'].isnull(), labs['REF_NORMAL_VALS'].str.contains(r'[<=]')==True), -999, labs.NEW_LOW) # solve cases where REF_LOW is like '<TAU', anything below TAU is normal => x is normal if -999 < x < TAU. 
labs['NEW_HIGH'] = np.where(labs['REF_NORMAL_VALS'].str.contains(r'[<=]'), labs['REF_NORMAL_VALS'].str.replace(r'[<=]',''), labs.NEW_HIGH) # NEW_LOW = -999, NEW_HIGH = TAU
# put REF_X into NEW_X
labs['NEW_LOW'] = np.where(labs.NEW_LOW.isnull(), labs['REF_LOW'], labs.NEW_LOW)
labs['NEW_HIGH'] = np.where(labs.NEW_HIGH.isnull(), labs['REF_HIGH'], labs.NEW_HIGH)
labs['NEW_HIGH'] = labs['NEW_HIGH'].astype(float)
labs['NEW_LOW'] = labs['NEW_LOW'].astype(float)    
labs['REF_UNIT'] = labs['REF_UNIT'].str.lower()

labs['RESULT_FLAG'] = 'Normal'
labs['RESULT_FLAG'] = np.where(labs['ORD_NUM_VALUE'] < labs.NEW_LOW, 'Low', labs['RESULT_FLAG'])
labs['RESULT_FLAG'] = np.where(labs['ORD_NUM_VALUE'] > labs.NEW_HIGH, 'High', labs['RESULT_FLAG'])
labs['RESULT_YEAR'] = labs.RESULT_DATE.dt.year
labs['RESULT_MONTH'] = labs.RESULT_DATE.dt.month

# Get total counts per month for each lab result type
labs['newcol'] = labs['GROUP']+'_'+ labs['RESULT_FLAG']
labs = labs[['AUTO_ID', 'RESULT_YEAR', 'RESULT_MONTH', 'newcol']]
labs['chk'] = 1
labs=labs.groupby(identifier_cols+['newcol']).chk.sum().unstack(-1).reset_index()
labs.to_csv('flatfiles/labs.csv', index=False)


# Get record of various diagnostic tests (related to GI)

In [80]:
"""
DEPRECATED - using old file
"""

# diagnostic = pd.read_csv(os.path.join(cfg.IN_DATA_PATH, 'deid_IBD_Registry_BA1951_Rad_Diagnostic_Tests_2018-07-05-12-19-00.csv'), parse_dates=['VISIT_DATE'])
# diag = diagnostic[diagnostic.ORDER_TYPE=='GI PROCEDURES']

# def onehot_flag(col_name, pat):
#     diag[col_name] = 0
#     diag.loc[diag.DESCRIPTION.str.contains(pat), col_name] += 1 

# col_pat = [('COLONOSCOPY', 'COLONOSCOP'), ('ENDOSCOPY', 'ENDOSCOP'), ('SIGMOIDOSCOPY', 'SIGMOIDOSCO'), ('ILEOSCOPY', 'ILEOSCOP'), ('ERCP', 'ERCP'), ('EGD', 'ESOPHOGASTRODUODENOSCOPY'), ('EGD', 'UPPER GI ENDO'), ('EGD','UPPER EUS'), ('ANO','ANOSCOPY'), ('EGD', 'ESOPHAGOSCOPY')]

# for c,p in col_pat:
#     onehot_flag(c, p)

# """
# DON't KNOW WHAT THIS IS FOR
# diag['GI_PROCEDURE'] = 0
# diag.loc[~diag[[c for c,_ in col_pat]].any(axis=1), 'GI_PROCEDURE'] = 1
# """

# diag['RESULT_YEAR'] = diag.VISIT_DATE.dt.year
# diag['RESULT_MONTH'] = diag.VISIT_DATE.dt.month
# diag = diag[['AUTO_ID','RESULT_YEAR', 'RESULT_MONTH']+[c for c,_ in col_pat].drop_duplicates()
# diag=diag.groupby(identifier_cols).sum().reset_index()
# diag.to_csv(opj(cfg.LONG_IN, 'gi_diagnostics.csv'), index=False)

Alternate Diagnostics

In [35]:
diag = pd.read_csv('deid_R3_84_BINION_PROCEDURE_DATA_2018_11_05.csv', parse_dates=['ORIG_SERVICE_DATE'])
diag['RESULT_YEAR'] = diag.ORIG_SERVICE_DATE.dt.year
diag['RESULT_MONTH'] = diag.ORIG_SERVICE_DATE.dt.month
diag.drop(diag.index[diag.AUTO_ID=='NO_MATCH'], axis=0, inplace=True)
diag.AUTO_ID = diag.AUTO_ID.astype(int)

def onehot_flag(col_name, pat):
    diag['DIAG_'+col_name] = 0
    diag.loc[diag.PROC_NAME.str.contains(pat) & (diag.PROC_GROUP_NAME=='DIAGNOSTIC RADIOLOGY'), 'DIAG_'+col_name] += 1 

col_pat = [('COLONOSCOPY', 'COLONOSCOP'), ('ENDOSCOPY', 'ENDOSCOP'), ('SIGMOIDOSCOPY', 'SIGMOIDOSCO'), ('ILEOSCOPY', 'ILEOSCOP'), ('ANO','ANOSCOPY')]

for c,p in col_pat:
    onehot_flag(c, p)

diag['DIAG_CT_ABPEL'] = np.where(np.logical_or(diag.PROC_NAME.str.contains('CT ABDOMEN'), diag.PROC_NAME.str.contains('CT PELVIS')), 1, 0)
diag = diag[IDENTIFIER_COLS+[x for x in diag.columns if x[:4]=='DIAG']]
diag = diag.groupby(identifier_cols, as_index=False).sum()
diag.sort_values(identifier_cols).to_csv('flatfiles/gi_diagnostics.csv', index=False)

# Surgeries

In [ ]:
surg = pd.read_csv('deid_R3_84_BINION_PROCEDURE_DATA_2018_11_05.csv', parse_dates=['ORIG_SERVICE_DATE'])
surg = surg[surg.AMOUNT>0] 

surg['RESULT_YEAR'] = surg.ORIG_SERVICE_DATE.dt.year
surg['RESULT_MONTH'] = surg.ORIG_SERVICE_DATE.dt.month
surg.drop(surg.index[surg.AUTO_ID=='NO_MATCH'], axis=0, inplace=True)
surg.AUTO_ID = surg.AUTO_ID.astype(int)

reqd_surg = surg[(surg.TYPE_OF_SERVICE=='20-SURGERY MA') & (surg.PROC_GROUP_NAME=='GASTROINTESTINAL/DIGESTIVE') & (~surg.PROC_NAME.str.contains('NEEDLE BIOPSY')) & (~surg.PROC_NAME.str.contains('APPENDECTOMY')) & (~surg.PROC_NAME.str.contains('GASTROSTOMY')) & (~surg.PROC_NAME.str.contains('ESOPH')) & (~surg.PROC_NAME.str.contains('CHOLECYSTOSTOMY')) & (~surg.PROC_NAME.str.contains('ANAST')) & (~surg.PROC_NAME.str.contains('TONGUE')) & (~surg.PROC_NAME.str.contains('LIVER')) & (~surg.PROC_NAME.str.contains('PLASTY')) & (~surg.PROC_NAME.str.contains('BILIARY')) & (~surg.PROC_NAME.str.contains('BILE')) & (~surg.PROC_NAME.str.contains('PAROTD')) & (~surg.PROC_NAME.str.contains('EXTRAHEP')) & (~surg.PROC_NAME.str.contains('PANCREA')) & (~surg.PROC_NAME.str.contains('SCLEROTX')) & (~surg.PROC_NAME.str.contains('GUM LESION'))]

surgeries = pd.concat([reqd_surg, 
            surg[surg.PROC_CODE.isin(['44205', '44212', '44210', '44211', '44207', '44206', '44204', '44158'])], # COLECTOMIES
            surg[surg.PROC_NAME.str.contains('CELIAC PLEXUS')]])

surgeries = surgeries.sort_values(['AUTO_ID', 'ORIG_SERVICE_DATE']).drop_duplicates(['AUTO_ID', 'ORIG_SERVICE_DATE'])
surgeries['ENC_PK_INJ'] = np.where(surgeries.PROC_NAME.str.contains('CELIAC PLEXUS'), 1, 0)
surgeries['ENC_SURGERY'] = (1 - surgeries['ENC_PK_INJ']).abs()

surgeries[IDENTIFIER_COLS+['ENC_SURGERY', 'ENC_PK_INJ']].groupby(IDENTIFIER_COLS, as_index=False).sum().to_csv('flatfiles/surgeries.csv', index=False)

# Get encounter information
- Flag for related/unrelated based on department/office (dermatology, ortho, opto, ent)

In [37]:
import re

enc_df = pd.read_csv("deid_IBD_Registry_BA1951_Office_Phone_Email_Encs_2018-07-05-09-43-22.csv", parse_dates=['CONTACT_DATE']).drop('Unnamed: 0', axis=1)
enc_df.ENC_TYPE_NAME = enc_df.ENC_TYPE_NAME.replace({'Telephone':'TEL', 'Office Visit':'OFF', 'Patient Email':'TEL', 'Procedure Visit':'PROC', 'New Patient Visit':'OFF', 'Consult':'OFF'})
enc_df=enc_df[enc_df['DEPT_NAME'].notnull()]


# related/unrelated
enc_df['unrelated'] = 0
bad_dept = re.compile(r'DERM |ORTHO |OPTOMETRY |ENT MERCY')
enc_df['unrelated'] = enc_df.DEPT_NAME.str.match(bad_dept).astype(int)

ENC = enc_df[['AUTO_ID', 'ENC_TYPE_NAME', 'CONTACT_DATE', 'unrelated']]
ENC = pd.get_dummies(ENC, columns=['ENC_TYPE_NAME'], prefix='ENC')
ENC['RESULT_YEAR'] = ENC.CONTACT_DATE.dt.year
ENC['RESULT_MONTH'] = ENC.CONTACT_DATE.dt.month



ENC = ENC[identifier_cols+[x for x in ENC.columns if x not in identifier_cols]]
ENC['unrelated'] = ENC['unrelated'].replace({0:'Related', 1:'Unrelated'})
ENC = ENC.groupby(['AUTO_ID', 'RESULT_YEAR', 'RESULT_MONTH', 'unrelated']).sum()
ENC = ENC.unstack(-1).reset_index().fillna(0)
ENC.columns = ENC.columns.map('_'.join).str.strip('_')
ENC.to_csv('flatfiles/encounters.csv', index=False)

# SIBDQ - not using for now

In [125]:
# sibdq = pd.read_csv(os.path.join(IN_DATA_PATH, 'deid_IBD_Registry_BA1951_SIBDQ_Pain_Questionnaires_2018-07-05-10-02-02.csv'), parse_dates=['CONTACT_DATE']).drop(['Unnamed: 0', 'ENC_TYPE'], axis=1).dropna(how='all')
# sibdq.drop(sibdq.index[sibdq.AUTO_ID=='NO_MATCH'], axis=0, inplace=True)
# sibdq.AUTO_ID = pd.to_numeric(sibdq.AUTO_ID).astype(int)
# sibdq['PAIN_TODAY'] = sibdq['PAIN_TODAY'].fillna('No')

# #compute missing total scores
# q_cols = [x for x in sibdq.columns if x.startswith('Q')]
# # parse responses as integers
# sibdq[q_cols] = sibdq[q_cols].fillna('0').applymap(lambda st: int(st[0]))
# # fill empty totals
# null_scores_ix = sibdq[sibdq.SIBDQ_TOTAL_SCORE.isnull()].index
# sibdq.loc[null_scores_ix, 'SIBDQ_TOTAL_SCORE'] = sibdq.loc[null_scores_ix][q_cols].sum(axis=1).astype(int)
# sibdq.SIBDQ_TOTAL_SCORE = sibdq.SIBDQ_TOTAL_SCORE.astype(int)
# sibdq['PAIN_TODAY'] = sibdq['PAIN_TODAY'].replace({'Yes':1, 'No':0})
# sibdq.to_csv(os.path.join(OUT_DATA_PATH, 'sibdq.csv'))

# HBI

In [38]:
hbi = pd.read_csv('deid_IBD_Registry_BA1951_Harvey_Bradshaw_Questionnaires_2018-07-05-07-57-02.csv', parse_dates=['CONTACT_DATE'])
hbi = hbi[['AUTO_ID', 'CONTACT_DATE', 'CROHNS_SCORE', 'UC_SCORE']]
hbi.AUTO_ID = pd.to_numeric(hbi.AUTO_ID).astype(int)
hbi['RESULT_YEAR'] = hbi.CONTACT_DATE.dt.year
hbi['RESULT_MONTH'] = hbi.CONTACT_DATE.dt.month
del hbi['CONTACT_DATE']

HBI = hbi.groupby(IDENTIFIER_COLS).mean().reset_index()
HBI.to_csv('flatfiles/hbi.csv', index=False)

# Disease Type

In [39]:
disease = pd.read_csv('deid_disease type and year of diagnosis.csv')
disease.DSTYPE = disease.DSTYPE.replace({'1':'CD', '2':'UC', '-2':'unk', '3':'unk'})
disease[['DS_CD', 'DS_UC']] = pd.get_dummies(disease.DSTYPE).drop('unk', axis=1)
disease['DS_AGE_DX'] = disease['AGE_OF_DX'].replace('#VALUE!', np.nan).astype(float).fillna(30).astype(int) # 30 is the average age
disease['DS_PREV_RESECTION'] = disease['BOWEL_RESECTION_PRIOR_2009'].replace({'(No Data)':'0'}).fillna(0).astype(int)
DS = disease[['AUTO_ID']+[x for x in disease.columns if x[:3]=='DS_']]
DS.to_csv('flatfiles/disease_type.csv', index=False)


# Charges

In [40]:
def get_inflation_data():
    #http://www.usinflationcalculator.com/inflation/current-inflation-rates/
    #I took it from the December column as it is explained on the website (last 12 months inflation)
    #year,inflation
    inflationrates={2005:3.4,
                    2006:2.5,
                    2007:4.1,
                    2008:0.1,
                    2009:2.7,
                    2010:1.5,
                    2011:3.0,
                    2012:1.7,
                    2013:1.5,
                    2014:0.8,
                    2015:0.7,
                    2016:2.1,
                    2017:2.1 } #last update: last charges made in 2018, so need to take into account only till 2017
    for (year,rate) in inflationrates.items():
        inflationrates[year]=rate*0.01+1
    
    #calculate coefficients    
    lastyear=np.max(list(inflationrates.keys()))
    inflation_coeff={lastyear+1:1.0}
    for year in range(lastyear, 2005, -1):
        inflation_coeff[year]=inflation_coeff[year+1]*inflationrates[year]
    return inflation_coeff

In [42]:
inflation_coeff = get_inflation_data()

# unrelated_ip CHARGES -- from Claudia
unrelated_ip = pd.read_csv('ibd_related_ip_charges.csv', parse_dates=['ADMISSION_DATE', 'DISCHARGE_DATE'])
unrelated_ip['PAT_MRN_ID'] = unrelated_ip['PAT_MRN_ID'].astype(int)
unrelated_ip['related'] = unrelated_ip['related'].replace({2:1})
unrelated_ip = unrelated_ip.merge(mrn_auto_map[['AUTO_ID', 'PAT_MRN_ID']], on='PAT_MRN_ID')


#INPATIENT
inpatient_charges_txn = pd.read_csv("deid_IP_Charges_Aug_2018_12_6_18.csv",thousands=r',', parse_dates=['ADMISSION_DATE', 'DISCHARGE_DATE']).drop('Unnamed: 0', axis=1)
inpatient_charges_txn.drop(inpatient_charges_txn.index[inpatient_charges_txn.AUTO_ID=='NO_MATCH'], axis=0, inplace=True)
inpatient_charges_txn.index=inpatient_charges_txn.index.astype(int)
#adjust for inflation
inpatient_charges_txn['INPATIENT_CHARGES']=inpatient_charges_txn.apply(lambda row: row['TOTAL_CHARGES']*inflation_coeff[row['DISCHARGE_DATE'].year], axis=1)
inpatient_charges_txn.AUTO_ID = inpatient_charges_txn.AUTO_ID.astype(int)

# inpatients must have >0 DAYS_ADMITTED
inpatient_charges_txn['DAYS_ADMITTED'] = (inpatient_charges_txn['DISCHARGE_DATE'] - inpatient_charges_txn['ADMISSION_DATE']).dt.days
inpatient_charges_txn.loc[inpatient_charges.DAYS_ADMITTED==0, 'DAYS_ADMITTED'] = 1


inpatient_charges = inpatient_charges_txn[['AUTO_ID', 'ADMISSION_DATE', 'DISCHARGE_DATE', 'DAYS_ADMITTED', 'INPATIENT_CHARGES']]
inpatient_charges['INPATIENT'] = 1
inpatient_charges.columns = ['AUTO_ID', 'ADMISSION_DATE', 'DISCHARGE_DATE', 'DAYS_ADMITTED', 'AMOUNT', 'INPATIENT']

unrelated_ip = unrelated_ip[['AUTO_ID', 'ADMISSION_DATE', 'DISCHARGE_DATE', 'related']]
inpatient_charges = inpatient_charges.merge(unrelated_ip, how='left',  on=['AUTO_ID', 'ADMISSION_DATE', 'DISCHARGE_DATE'])
inpatient_charges['unrelated'] = (inpatient_charges['related'].replace({2:0}) - 1).abs().fillna(0)
inpatient_charges.drop(['related'], inplace=True, axis=1)
inpatient_charges.to_csv('flatfiles/inpatient_charges.csv')

FileNotFoundError: [Errno 2] File b'ibd_related_ip_charges.csv' does not exist: b'ibd_related_ip_charges.csv'

In [79]:
# # OUTPATIENT
outpatient = pd.read_csv(os.path.join(cfg.IN_DATA_PATH, "deid_OP_charges_2018.csv"), parse_dates=['ORIG_SERVICE_DATE'])
outpatient = outpatient[outpatient['AMOUNT']>0]
outpatient.drop(outpatient[outpatient.AUTO_ID=='NO_MATCH'].index, inplace=True)
outpatient = outpatient[outpatient.DX.notnull() & outpatient.DX!='']
outpatient.AUTO_ID=outpatient.AUTO_ID.astype(int)




# TAGGING UNRELATED PROCEDURES ((422943, 9), (383395, 9), (357001, 10))
# =======================================================================

# Method 0: No filter
# outpatient_charges_transactions0 = outpatient

# Method 1: USe Claudia-provided procedures_to_exclude.csv as mask
bad_proc = pd.read_csv(os.path.join(cfg.IN_DATA_PATH, 'procedures_to_exclude.csv'))
outpatient_charges_transactions1 = outpatient.merge(bad_proc, how='left', on='PROC_NAME').drop_duplicates()
 

# Method 2: Use Suraj-approximated ICD-classes filter in bad_code_ranges
# def separate_dx_codes(opc):
#     opc1 = opc[opc.DX.notnull() & opc.DX.str.contains(',')]
#     opc2 = opc[~opc.index.isin(opc1.index)]
#     dx = opc1.DX.str.split(', ').apply(pd.Series).stack()
#     dx = dx.droplevel(-1)
#     dx.name = 'DX'
#     del opc1['DX']
#     opc1 = opc1.join(dx)
#     opc1 = opc1[opc2.columns]
#     opc = pd.concat([opc1, opc2]).reset_index()
#     return opc
# outpatient = separate_dx_codes(outpatient)
# opc = outpatient[outpatient.DX!='']
# bad_code_ranges = [(630, 780), (460, 520), (280, 290)]
# opc_num = opc[~(opc.DX.str.startswith('V') | opc.DX.str.startswith('E'))]

# Method 3: Specify unrelated PROC_GROUP_NAME
bad_proc_grp = ['FEMALE GENITAL & REPRODUCTIVE', 'INTEGUMENTARY/DERMATOLOGY', 'MUSCLE/SKELETAL SYSTEM',\
            'NERVOUS SYSTEM & NEURO STUDIES', 'MATERNITY CARE & DELIVERY', 'EAR, NOSE & THROAT', 'DENTAL', 'DME',\
           'EYE & OCULAR', 'MALE GENITAL', 'OTHER', 'PHYSICAL MEDICINE & REHAB', 'PULMONARY', 'SUPPLIES', 'URINARY/RENAL SYSTEM']

# =======================================================================

outpatient_charges_transactions = outpatient_charges_transactions1
outpatient_charges_transactions['AMOUNT']=outpatient_charges_transactions.apply(lambda row: row['AMOUNT']*inflation_coeff[ row['ORIG_SERVICE_DATE'].year ], axis=1) #adjust by inflation
outpatient_charges_transactions.loc[outpatient_charges_transactions.PROC_GROUP_NAME.isin(bad_proc_grp), 'unrelated'] = 1
outpatient_charges_transactions['unrelated'] = outpatient_charges_transactions['unrelated'].fillna(0)
outpatient_charges_transactions.to_csv(os.path.join(cfg.OUT_DATA_PATH, 'outpatient_charges_txn.csv'))

outpatient_charges_transactions['ADMISSION_DATE'] = outpatient_charges_transactions['ORIG_SERVICE_DATE']
outpatient_charges_transactions['DISCHARGE_DATE'] = outpatient_charges_transactions['ORIG_SERVICE_DATE']
outpatient_charges_transactions['DAYS_ADMITTED'] = 0
outpatient_charges_transactions['INPATIENT'] = 0
op = outpatient_charges_transactions[['AUTO_ID', 'ADMISSION_DATE', 'DISCHARGE_DATE', 'DAYS_ADMITTED', 'AMOUNT', 'unrelated', 'INPATIENT']]
op.to_csv(os.path.join(cfg.OUT_DATA_PATH, 'outpatient_charges.csv'))


In [45]:
tot = pd.concat([inpatient_charges, op], ignore_index=True).sort_values(['AUTO_ID', 'DISCHARGE_DATE'])
tot['RESULT_YEAR'] = tot.DISCHARGE_DATE.dt.year
tot['RESULT_MONTH'] = tot.DISCHARGE_DATE.dt.month
tot['CUMSUM'] = tot.groupby(['AUTO_ID', 'unrelated']).AMOUNT.cumsum()
tot['unrelated'] = tot['unrelated'].replace({0:'RELATED', 1:'UNRELATED'})
tot['INPATIENT'] = tot['INPATIENT'].replace({0:'OP', 1:'IP'})
tot.to_csv(os.path.join(cfg.OUT_DATA_PATH, 'total_charges.csv'), index=False)

In [58]:
aggregates = tot.groupby(['AUTO_ID', 'RESULT_YEAR', 'RESULT_MONTH', 'INPATIENT', 'unrelated']).AMOUNT.sum()
agg = aggregates.unstack([-1, -2]).reset_index().fillna(0)
agg.columns = agg.columns.map('_'.join).str.strip('_')
agg.to_csv(os.path.join(cfg.LONG_IN, 'charges.csv'), index=False)

In [60]:
agg

,AUTO_ID,RESULT_YEAR,RESULT_MONTH,RELATED_OP,UNRELATED_OP,RELATED_IP,UNRELATED_IP
0,0,2009,8,138.383465,0.000000,0.0,0.0
1,0,2009,9,79.746403,0.000000,0.0,0.0
2,0,2010,1,1050.556892,0.000000,0.0,0.0
3,0,2010,2,0.000000,184.989366,0.0,0.0
4,0,2010,3,173.570269,0.000000,0.0,0.0
...,...,...,...,...,...,...,...
87114,3140,2017,3,134.782210,0.000000,0.0,0.0
87115,3140,2017,6,105.163000,0.000000,0.0,0.0
87116,3140,2018,8,103.000000,0.000000,0.0,0.0
87117,3141,2017,9,456.397210,86.785000,0.0,0.0
